In [1]:
from langgraph.graph import StateGraph, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver

class State(TypedDict):
    value: str

memory = MemorySaver()

def node_a(state: State):
    print("Node A")
    return Command(
        goto =  "node_b",
        update = {
            "value": state["value"] + "A",
        }
    )

def node_b(state: State):
    print("Node B")
    input = interrupt("Do you want to go to C or D? Type C/D")

    print(f"Human review values: {input}")
    if(input == "C"):
        return Command(
            goto ="node_c",
            update = {
                "value": state["value"] + "B"
            }
        )
    elif(input == "D"):
        return Command(
            goto = "node_d",
            update = {
                "value": state["value"] + "B"
            }
        )
    else:
        print("Invalid input, going to endpoint")
        return Command(
            goto = END,
            update = {
                "value": state["value"] + "InvalidInput"                                                                                                         
            }
        )                                                  


def node_c(state: State):
    print("Node_C")
    return Command(
        goto = END,
        update = {
            "value": state["value"] + "C"
        }
    )

def node_d(state: State):
    print("Node_D")
    return Command(
        goto = END,
        update = {
            "value": state["value"] + "D"
        }
    )

graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)    
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)   
graph.set_entry_point("node_a")

app = graph.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}

initial_state= {
    "value": ""
    }
response = app.invoke(initial_state, config, stream_mode = "updates")

print(response)


Node A
Node B
[{'node_a': {'value': 'A'}}, {'__interrupt__': (Interrupt(value='Do you want to go to C or D? Type C/D', resumable=True, ns=['node_b:247c6aff-a99c-5aaa-a0f7-cf951ae26f94']),)}]


In [2]:
print(app.get_state(config).next)

('node_b',)


In [3]:
second_result = app.invoke(Command(resume="C"), config=config, stream_mode="updates")
second_result

Node B
Human review values: C
Node_C


[{'node_b': {'value': 'AB'}}, {'node_c': {'value': 'ABC'}}]